# ReGen Worker - Kaggle Branch
### Remember to first set your API KEY and worker name
#### You can serve different models, simply change the name(s) in models_to_load to match the model(s) you want, you can check either https://aqualxx.github.io/stable-ui/workers, in the models tab, https://tinybots.net/artbot/info/models, or https://aihorde.sitew3.com/#/models, just copy paste the name (it's set to AlbedoBase XL (SDXL) by default).
#### You can also change max_power and see how high you can go, it's set to 32 by default

In [ ]:
#0.- This cell will set the variables and rerun the worker if you stopped it, but only if everything else was installed already

#### For right now, THESE are the only variables that we care about
worker_name = ""
api_key = ""
civitai_token = "cae554bcc138d97a9323856c2dee1158"
max_power = 32
max_batch = "5"

# Set the model(s) down here
models_to_load = ["AlbedoBase XL (SDXL)"]
queue_size = 2
max_threads = 1

# I have not tested how this works at all, I'm just putting it together from the old notebook
# The 2 gpus version DOES NOT support batches, do not ask me about it
# I don't know if/how it will work with outside models, best not to try it
gpu_2x_version = False
if (gpu_2x_version):
    # if using the 2 gpus version
    models_to_load = ["AlbedoBase XL (SDXL)", "Anything Diffusion"]
    queue_size = 3
    max_threads = 2


safety_on_gpu = True
allow_lora = True
nsfw = True
censor_nsfw = False
####For right now, THESE are the only variables that we care about

#these are not used yet
auto_dual_gpu = True
models_to_pin = ["AlbedoBase XL (SDXL)", "Anything Diffusion"]
gpus_to_use = [0, 1]

horde_url = "https://aihorde.net"
allow_painting = False
dynamic_models = False
models_to_skip = ["stable_diffusion_inpainting", "stable_diffusion_2.1",  "stable_diffusion_2.0"]
allow_post_processing = False
priority_usernames = []
blacklist = []
censorlist = []
allow_img2img = True
allow_controlnet = False
allow_unsafe_ip = True
number_of_dynamic_models = 0
max_models_to_download = 10
forms = ["caption","nsfw","interrogation","post-process"]


# To use unsupported/deprecated models (Segmind, Van Gogh Diffusion, etc)
# You need to set this next variable to True if using this, otherwise make sure it's False
outside_model = False
# The name of the model you are going to use, this is only so you know what model it is
outside_model_name = "Pony V6, SD 1.5"
# Part of the file name when downloaded; can't remember if it's actually necessary after everything I changed, but I can't be bothered to check/remove it
model_file_name = "pony"
# The baseline of the model you are using, 1.5 or XL
baseline = "SD 1.5" # ["SD 1.5", "SD XL"]
# The download url, tested for both huggingface and civitai, so it should be fine
outside_model_download_url = "https://civitai.com/api/download/models/297064?type=Model&format=SafeTensor&size=pruned&fp=fp16"

if("PickleTensor" in outside_model_download_url or ".ckpt" in outside_model_download_url):
    file_extension = ".ckpt"
if("SafeTensor" in outside_model_download_url or ".safetensors" in outside_model_download_url):
    file_extension = ".safetensors"

civitai = "civitai.com/api/download/models"
if (civitai in outside_model_download_url):
    outside_model_download_url = outside_model_download_url.split("?")[0] + "?token=" + civitai_token

mask15_ckpt = {
    'model_name': "Anything Diffusion",
    'file_name': "Anything-Diffusion.ckpt",
    'sha256': "633c153d96230355efb4230da6ae2e3ba85b084b93c89eb88cb1118d6cc06cef *Anything-Diffusion.sha256",
    'sha_name': "Anything-Diffusion.sha256"
}
mask15_safe = {
    'model_name': "Dreamshaper",
    'file_name': "Dreamshaper.safetensors",
    'sha256': "879db523c30d3b9017143d56705015e15a2cb5628762c11d086fed9538abd7fd *Dreamshaper.sha256",
    'sha_name': "Dreamshaper.sha256"
}
maskxl_safe = {
    'model_name': "AlbedoBase XL (SDXL)",
    'file_name': "albedo_base_xl.safetensors",
    'sha256': "1718B5BB2DA1EF4815FEE8AF8A7FC2FA8AB8F467B279EDED4D991EA0CCE59A6D *albedo_base_xl.sha256",
    'sha_name': "albedo_base_xl.sha256"
}
if (baseline == "SD XL"):
    mask = maskxl_safe
if (baseline == "SD 1.5"):
    if(file_extension == ".ckpt"):
        mask = mask15_ckpt
    if(file_extension == ".safetensors"):
        mask = mask15_safe

if (outside_model and outside_model_name and outside_model_download_url):
    models_to_load = []
    models_to_load = [mask['model_name']]
    print ("Running model not supported by the Horde (non-Customizer Role)")
    print (f'''Model: {outside_model_name:}, Mask: {mask['model_name']}, format: {mask['file_name'].split(".")[1]}''')
# To use unsupported/deprecated models (Segmind, Van Gogh Diffusion, etc)


selected_models = set(list(models_to_load))
print (f'Running: {models_to_load}')


current_path = "/kaggle/working/"
worker_path = current_path + "horde-worker-reGen/"
bridgeData_file = worker_path + "bridgeData.yaml"

# To toggle Maintenance Mode ON/OFF
# Maintenance Mode will be turned ON automatically if using outside models
Maintenance_Mode = False


import os

if not os.path.exists(bridgeData_file):
    print ("bridgeData.yaml file not found. Proceed to install the worker.")
else:
    print ("bridgeData.yaml file found. Recreating bridgeData.yaml and restarting the worker.")
    create_yaml()
    download_models_aria2c()

    if (Maintenance_Mode):
        Maintenance_Mode = True
    else:
        if (outside_model and outside_model_name and outside_model_download_url):
            Maintenance_Mode = True
        else:
            Maintenance_Mode = False
    Set_Maintenance_Mode_function(Maintenance_Mode)

    #!cd /kaggle/working
    #!source ../regen/bin/activate;python download_models.py
    !cd /kaggle/working
    !source ../regen/bin/activate;python run_worker.py -vv


In [ ]:
# Get the Worker OUT of Maintenance Mode
#This cell is to get the worker out of maintenance before starting, since some people don't know how or where to do it

import subprocess
import json
import re
import requests

def Set_Maintenance_Mode_function(Maintenance_Mode):

    i = False #Worker does not match/exist flag

    command_find_user = f'''curl -X 'GET' \
              '{horde_url}/api/v2/find_user' \
              -H 'accept: application/json' \
              -H 'apikey: {api_key}' \
              -H 'Client-Agent: unknown:0:unknown'
            '''

    find_user_response = subprocess.check_output(command_find_user, shell=True).decode('utf-8')
    data_api_key = json.loads(find_user_response)

    if not ("message" in find_user_response):
        for worker_id in data_api_key['worker_ids']:
            #print(worker_id)

            command_find_worker = f'''curl -X 'GET' \
                      '{horde_url}/api/v2/workers/{worker_id}' \
                      -H 'accept: application/json' \
                      -H 'apikey: {api_key}' \
                      -H 'Client-Agent: unknown:0:unknown'
                    '''
            find_worker_response = subprocess.check_output(command_find_worker, shell=True).decode('utf-8')
            data_worker = json.loads(find_worker_response)
            if (worker_name == data_worker['name']):
                print(f'''Worker name: {worker_name}, Worker ID: {worker_id}''')
                i = True

                worker_uuid = worker_id

                if (Maintenance_Mode):
                  set_maintenance_mode = "true"
                else:
                  set_maintenance_mode = "false"

                command_maintenance_mode = f'''curl -X 'PUT' \
                  '{horde_url}/api/v2/workers/{worker_id}' \
                  -H 'accept: application/json' \
                  -H 'apikey: {api_key}' \
                  -H 'Client-Agent: unknown:0:unknown' \
                  -H 'Content-Type: application/json' \
                  -d'''+ r''' '{
                  "maintenance":''' + f''' {set_maintenance_mode}
                ''' + r'''}'
                '''

                maintenance_mode_response = subprocess.check_output(command_maintenance_mode, shell=True).decode('utf-8')
                print(maintenance_mode_response)

        if not i:
          print(f'''No worker with the name: {worker_name}, can't change Maintenance Mode. A new worker will be created with that name.''')
          worker_id = 0
    else:
        print(f'''The API key "{api_key}" does not exist''')

Set_Maintenance_Mode_function(Maintenance_Mode = False)


In [ ]:
#1.- Virtual Environment

!python -m virtualenv regen


In [ ]:
#2.- Remove the worker if it exists, then, Clone the regen worker

!cd /kaggle/working;rm -r horde-worker-reGen

if not (gpu_2x_version):
    # if using the 1 gpu version
    !cd /kaggle/working;git clone https://github.com/Haidra-Org/horde-worker-reGen.git
else:
    # if using the 2 gpus version
    !cd /kaggle/working;git clone https://github.com/Haidra-Org/horde-worker-reGen.git --branch kaggle --single-branch


In [ ]:
#3.- Install requirements

if not (gpu_2x_version):
    # if using the 1 gpu version
    !source regen/bin/activate;pip install -r .\/horde-worker-reGen/requirements.txt
else:
    # if using the 2 gpus version
    !source regen/bin/activate;pip install -r .\/horde-worker-reGen/requirements.118.txt
    %cd /kaggle/working/
    !source regen/bin/activate;pip install -r .\/horde-worker-reGen/requirements.hordelib.txt
    !source regen/bin/activate;pip install hordelib==2.1.1 --no-deps


In [ ]:
#4.- Create .yaml config file

import os

%cd $worker_path
print ("Creating bridgeData.yaml file.")

def create_yaml():
    
    from yaml import load, dump

    def make_yaml_sublist(list_to_convert: list[str]):
        sublist_yaml = dump(list_to_convert)
        sublist_yaml = "\n" + sublist_yaml
        return sublist_yaml


    data = f"""horde_url: "{horde_url}"
api_key: "{api_key}"
civitai_api_token: "{civitai_token}"
gpus_to_use: {gpus_to_use}
priority_usernames: []
max_threads: {max_threads}
queue_size: {queue_size}
max_batch: {max_batch}
safety_on_gpu: {safety_on_gpu}
#auto_dual_gpu: {auto_dual_gpu}, no longer necessary, should be ignored like this
require_upfront_kudos: false
dreamer_name: "{worker_name}"
max_power: {max_power}
nsfw: {nsfw.__str__().lower()}
censor_nsfw: {censor_nsfw}
blacklist: {blacklist}
censorlist: {censorlist}
allow_img2img: {allow_img2img.__str__().lower()}
allow_painting: {allow_painting.__str__().lower()}
allow_unsafe_ip: true
allow_post_processing: {allow_post_processing.__str__().lower()}
allow_controlnet: false
allow_lora: {allow_lora.__str__().lower()}
max_lora_cache_size: 10
dynamic_models: false
number_of_dynamic_models: 0
max_models_to_download: 10
stats_output_frequency: 30
cache_home: "./"
always_download: true
temp_dir: "./tmp"
disable_terminal_ui: True
vram_to_leave_free: "80%"
ram_to_leave_free: "80%"
disable_disk_cache: false
models_to_load: {make_yaml_sublist(models_to_load)}
models_to_pin: {make_yaml_sublist(models_to_load)}
models_to_skip: {make_yaml_sublist(models_to_skip)}
suppress_speed_warnings: false
forms:
- "caption"
- "nsfw"
- "interrogation"
- "post-process"
"""

    with open(bridgeData_file, "w") as text_file:
        text_file.write(data)
    
    print ("bridgeData.yaml file created.")

create_yaml()


In [ ]:
#5.- Symlinks for models and loras

!mkdir /kaggle/tmp
!mkdir /kaggle/tmp/compvis
!mkdir /kaggle/tmp/lora


!ln -s /kaggle/tmp/compvis/ /kaggle/working/horde-worker-reGen/compvis
!ln -s /kaggle/tmp/lora/ /kaggle/working/horde-worker-reGen/lora


In [ ]:
# Download all models with aria2c

model_reference = "/kaggle/working/stable_diffusion.json"

# download the model reference if necessary
import os
if os.path.exists(model_reference):
    print("Model Reference exists.")
else:
    print("Downloading Model Reference.")
    !wget https://raw.githubusercontent.com/Haidra-Org/AI-Horde-image-model-reference/main/stable_diffusion.json -O /kaggle/working/stable_diffusion.json

# install aria2c if necessary
import subprocess
try:
    # Run the command
    output = subprocess.check_output(["which", "aria2c"])
    # unnecessary to do this next step, but whatever, I can use the output from above with this
    output = output.decode("utf-8")
    print("aria2c is installed.")
except:
    print(f"Installing aria2c.")
    !apt-get install -y aria2



# function to look in the model reference for the selected models
import json
def find_matching_entries(filename, key, value_list):
    """
    Finds all entries in a JSON file that match a given key and value list.

    Args:
      filename: The path to the JSON file.
      key: The key to search for.
      value_list: A list of values to match.

    Returns:
      A list of entries that match the criteria.
    """

    with open(filename, 'r') as f:
        data = json.load(f)

    matching_entries = []
    for entry in data.items():
        #print(entry)
        if entry[0] in value_list:
            matching_entries.append(entry)

    return matching_entries


# function to download the models with aria2c
def download_models_aria2c():

    # Example usage
    filename = model_reference
    key = "name"
    value_list = models_to_load
    # call the function
    matching_entries = find_matching_entries(filename, key, value_list)

    # download the model(s) and create the hash file(s)
    print(models_to_load)
    %cd $worker_path
    # Print the matching entries
    for entry in matching_entries:
        # add civitai api key to the download url
        if ("civitai" in entry[1]["config"]['download'][0]['file_url']):
            entry[1]["config"]['download'][0]['file_url'] = entry[1]["config"]['download'][0]['file_url'].split("?")[0] + "?token=" + civitai_token
        # extracting the required variables (url, name, sha)
        # download link
        # if all 3 outside model variables & match is the mask -> download link = outside model download url
        if (outside_model and outside_model_name and outside_model_download_url and entry[0] in mask['model_name']):
            print(f"Outside model: {outside_model_name}")
            download_link = outside_model_download_url
            # create file to mention the swap
            swap_file = outside_model_name + " --- " + mask['model_name']
            swap_path = "/kaggle/working/horde-worker-reGen/compvis/" + mask['model_name'] + ".outside"
            !echo {swap_file} > $swap_path
        else:
            download_link = entry[1]["config"]['download'][0]['file_url']

        print(download_link)
        # file name
        file_name = entry[1]["config"]['download'][0]['file_name']
        print(file_name)
        file_path = "compvis/" + file_name
        # sha256
        sha_name = file_name.split(".")[0] + ".sha256"
        sha = entry[1]["config"]['files'][0]['sha256sum']
        sha = sha + " *" + sha_name
        sha_path = "/kaggle/working/horde-worker-reGen/compvis/" + sha_name
        print(sha)
        # donwnload model and create sha file
        !aria2c -c -x 16 -s 16 -p $download_link -o $file_path
        !rm $sha_path
        !echo {sha} > $sha_path

download_models_aria2c()


In [ ]:
#6.- Run download models
# Make sure you have the correct path based on any `cd` commands above
!cd /kaggle/working

!source ../regen/bin/activate;python download_models.py


In [ ]:
#7.- Run the worker
# Make sure you have the correct path based on any `cd` commands above

if (outside_model and outside_model_name and outside_model_download_url):
    Maintenance_Mode = True
Set_Maintenance_Mode_function(Maintenance_Mode)

!cd /kaggle/working

!source ../regen/bin/activate;python run_worker.py -vv
